In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from openai.env file
load_dotenv("openai.env")
# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
# os.environ["OPENAI_API_KEY"] = api_key
# os.environ["OPENAI_API_BASE"] = api_base

In [5]:
! pip show openai
! pip show langchain
! pip show langchain-openai

Name: openai
Version: 1.23.1
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: doctran, langchain-openai
Name: langchain
Version: 0.1.16
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages
Requires: aiohttp, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Name: langchain-openai
Version: 0.1.3
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://github.com/langchain-ai/langchain
Author: 
Autho

# Embed_documents - 文档向量化
<hr>

In [12]:
from langchain_openai import OpenAIEmbeddings

e_model = OpenAIEmbeddings()
ebeddings = e_model.embed_documents(
    [
        "你好",
        "你好啊",
        "你叫什么名字?",
        "我叫王大锤",
        "很高兴认识你大锤",
    ]
)
len(ebeddings), len(ebeddings[0])
for eb in ebeddings:
    print(eb[:3])

[0.00020955622160252435, -0.006233604612335191, -0.0024306937584522587]
[-0.0021001850192939645, -0.012442666069256553, 0.0036603687223206584]
[0.002229126507444422, -0.00382731204210948, 0.016356849100074573]
[-0.020926184912547125, -0.011347020954711991, -0.006701486172475551]
[-0.0028738629242587603, -0.019952275300050004, 0.010649075967061414]


# embed_query - 问题向量化
<hr>

In [11]:
embedded_query = e_model.embed_query("这段对话中提到了什么名字?")
embedded_query[:5]

[0.0037302004841590695,
 0.0008163917854050834,
 0.02971025758721624,
 -0.006974423807729123,
 -0.025034373762037175]

# 嵌入向量缓存

<hr>

In [13]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

u_embeddings = OpenAIEmbeddings()
fs = LocalFileStore("./cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    u_embeddings,
    fs,
    namespace=u_embeddings.model,
)
list(fs.yield_keys())

[]

In [15]:
#加载文档，切分文档，将切分文档向量化病存储在缓存中
raw_documents = TextLoader("testfile/letter.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=620, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

In [ ]:
! pip install faiss-cup

In [21]:
from langchain.vectorstores import FAISS
%timeit -r  1 -n 1 db= FAISS.from_documents(documents,cached_embeddings)

11 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [18]:
#查看缓存中的键
list(fs.yield_keys())

['text-embedding-ada-002c63ea318-3b5d-533b-960b-46434f8b3c22',
 'text-embedding-ada-002f05b40fb-a095-546e-9c5d-49e069720828',
 'text-embedding-ada-0024250f053-4b1e-5c34-927d-a7857749217f',
 'text-embedding-ada-002b0c54c27-a009-50b4-9ccc-661d5478b195',
 'text-embedding-ada-002e94acbbe-7d17-5331-8310-4e37bdc56d31',
 'text-embedding-ada-0029286d74c-b3fc-56ff-8b08-9071a193f724']